<img src="./images/logo.png" alt="Drawing" style="width: 500px;"/>

# **Exercise 2:** Connecting and Querying Data Sources with EzPresto

This exercise will be introduce **EzPresto**, HPE Ezmeral's supercharged Presto distribution on **HPE Ezmeral Unified Analytics Software**. EzPresto is an SQL query engine based on the open-source query engine PrestoDB that is optimized to run federated queries across various data sources. Enterprise applications such as Supersets, Tableau, Power BI, and data processing engines (such as Spark), can leverage EzPresto for rapid query performance and prompt insights through federated data access. 

EzPresto is the underlying driver behind all of what you will learn in this exercise, in which you will:

- Connect Data Sources to HPE Ezmeral Unified Analytics.
- Run SQL queries using the Unified Analytics Query Editor.
- Learn about HPE's custom Jupyter Magic commands.
- Run SQL queries on connected Data Sources directly from within a notebook cell.

By the end of this exercise, you will be proficient in directly or indirectly leveraging EzPresto to streamline your data analytics workflows. 

Let's get started!

## **Contents:**
insert ToC here

<div class="alert alert-block alert-danger">
<b>Important:</b> This exercise requires the completion of Exercise 1: Exploring Retail Data with Apache Spark.</div>

## **1. Connecting a Data Source in Unified Analytics**

HPE Ezmeral Unified Analytics allows users to connect multiple types of internal and external data sources - from SQL servers to Snowflake, Terradata and Oracle databases - and make the files, objects and tables within them available to any tool or application running on Unified Analytics.

In this section, you will learn how to make a data connection using the Delta Tables you created in Exercise 1. 

### Connect Delta Tables as Data Source using Hive.

Let's take those Delta Tables you created in Exercise 1 and make them available to other applications in Unified Analytics by making them a **Data Source**. This will allow you to use EzPresto to turn them into datasets later in the exercise. 

1. Navigate back to the Unified Analytics dashboard.
1. In the sidebar navigation menu, select `Data Engineering` > `Data Sources`.
1. Under the `Structured Data` tab, click `Add New Data Source`.


<img src="./images/exercise2/datasources.png" alt="Drawing" style="width: 600px;"/>

4. Click `Create Connection` under **Hive**. 
- **Name**: `retail`
- **Hive Metastore:** `discovery`
- **Data Dir:** `file:/data/shared/retail-delta/data`
- **File Type:** `PARQUET`
5. Click `Connect`.



![image.png](./images/exercise2/connect-dl.png)

6. Under the `Structured Data` tab, you will now see your connected data source. 

### Viewing and Querying Data from Data Sources

Now that our Delta Tables are available via a Data Source, we can leverage the native data tools within Unified Analytics to run queries and create datasets.

1. Click `Query using Data Catalog` under the newly created data source.

![image.png](./images/exercise2/tile.png)

2. Under `Connected Data Sources`, look for the `data` group.
3. Under the `data` group, check the `default` box. 
4. Select the datasets for all three countries. 


![image.png](./images/exercise2/datacatalog.png)

5. Click `Selected Datasets` in the top left corner.
6. Click `Query Editor`.

Here, you are introduced to the the **Unified Analytics Query Editor** where you can directly query data sources from specific datasets and data tables - all from within the Unified Analytics user interface! 

![image.png](./images/exercise2/QueryEditor.jpg)

7. Next, we're going to run an SQL query which will combine the data from our three tables (czech, germany, and swiss) in the retail.default schema. This will merge all columns from the czech and germany tables, and select specific columns from the swiss table whilst also applying a transformation to the country column. We'll also limit our final result set to 1000 rows. And all in a nice UI!

    Paste the following SQL Query into the `SQL Query` field and 
    click `Run`.

```sql
SELECT * FROM retail.default.czech UNION ALL SELECT * FROM retail.default.germany UNION ALL ( SELECT PRODUCTID , PRODUCT , TYPE , UNITPRICE , UNIT , QTY , TOTALSALES , CURRENCY , STORE , (CASE WHEN (country = 'Swiss') THEN 'Switzerland' ELSE country END) COUNTRY , YEAR FROM retail.default.swiss ) LIMIT 1000

```

8. Expand the resulting query to visually validate it.
9. Under `Actions` (top-left corner of the table), click `Save as View`.

![image.png](./images/exercise2/query-results.png)

10. Name the View `retail`. 
11. We'll want to save the schema of this new table as a Custom Schema. Under `Schema`, select `+ Add new schema` and name it `retailschema`.

![image.png](./images/exercise2/save-as-view.png)

## **2. Jupyter Magic Commands on HPE Ezmeral Unified Analytics**

Jupyter Notebooks Magic functions, also known as magic commands or magics, are commands that you can execute within a code cell.   
Magics are not code of any language, but are shortcuts that extends the capabilities of a notebook. 

There are two types of magic commands - **Line** and **Cell** magic commands:

**Line magic** commands do not require a cell body and start with a single % character.  
**Cell magic** commands start with %% and require additional lines of input (a cell body). 

### HPE Ezmeral Magic Commands 
**HPE Ezmeral Unified Analytics Software** supports both Line and Cell magic commands and includes custom commands that allow for users to interact with other tools native to Unified Analytics directly within notebooks.

We can check out the full list of custom HPE magic commands by running `%command`.

In [8]:
# The %commands command lists the magic commands and SDKs that are customized by Hewlett Packard Enterprise and are available in this notebook.
%commands

HTML(value='\n    <table style="border-collapse: collapse; border: 1px solid black; width: 100%;">\n        <t…

### Updating the Cached JWT Token

In a Jupyter notebook, a JWT token (JSON Web Token) is a compact and URL-safe means of representing authentication information to be transferred between other notebook servers or external applications. It is commonly used for securely authenticating and authorizing users within Jupyter environments, allowing them to access resources and execute code while ensuring their identity and permissions are properly validated.

When working in Jupyter notebooks for long durations, particularly when making calls to other applications, the JWT token can expire and result in an error when attempting to make calls. This is particularly relevant for working on a Jupyter notebook within **HPE Ezmeral Unified Analytics**, which provides users to leverage a plethora of external tools within the notebook (Such as Spark, Livy and Presto).

If you encounter a JWT token expiration error while running cells in a Jupyter notebook, you can resolve it by running the `%update_token` magic command.  
This function updates the JWT in environment variables and any other locations where the token is utilized. 
  
Ideally, it is good practice to refresh the token prior to making external connections. Some examples relevant to the Smart Retail Experience demo include:

- Authentication when establishing a connection with PrestoDB.
- Authentication with local s3 minio object storage.  
- Authentication with KServe external API.  

In [9]:
%update_token

Switching to connection presto://ezpresto-svc-locator.ezpresto.svc.cluster.local:8080

Token suseccfully refreshed.


### Directly interacting with connected SQL databases using the SQL Magic Command

Using the `%sql` magic command, you can directly query SQL databases from Data Sources you have connected to **HPE Ezmeral Unified Analytics Software** from within Jupyter notebook cells! When you run the notebook cell containing `%sql` and your SQL query, the magic command sends the query to the database, runs the query, and retrieves the result.

This is made possible by the native integration of EzPresto into Unified Analytics. **However, the Data Source must be made publicly available.**

To change the access of a Data Source from `private` to `public`:

1. Navigate back to the Unified Analytics dashboard.
1. In the sidebar navigation menu, select `Data Engineering` > `Data Sources`.
1. Under the three dots in the top corner of the Data Source of interest, click `Change to public access`.

![image.png](./images/exercise2/public-access.png)

<div class="alert alert-block alert-danger">
<b>Important:</b> Wait until a confirmation message appears stating that the source is publicly available.
</div>

Let's try the `%sql` magic command to interact with our Delta Tables. 

In [5]:
%sql select * from retail.retail.czech limit 10

Running query in 'presto://ezpresto-svc-locator.ezpresto.svc.cluster.local:8080'

select * from retail.default.czech limit 10
RuntimeError: (pyhive.exc.DatabaseError) {'message': 'The column type of table default.czech is declared as type string, but the Parquet file (file:/data/shared/retail-delta/data/czech/part-00000-7a9f285f-a3ea-4a9a-932b-e45f3de1a4ca-c000.snappy.parquet) declares the column as type INT32', 'errorCode': 16777224, 'errorName': 'HIVE_PARTITION_SCHEMA_MISMATCH', 'errorType': 'EXTERNAL', 'boolean': False, 'failureInfo': {'type': 'com.facebook.presto.spi.PrestoException', 'message': 'The column type of table default.czech is declared as type string, but the Parquet file (file:/data/shared/retail-delta/data/czech/part-00000-7a9f285f-a3ea-4a9a-932b-e45f3de1a4ca-c000.snappy.parquet) declares the column as type INT32', 'suppressed': [], 'stack': ['com.facebook.presto.hive.parquet.ParquetPageSourceFactory.getParquetType(ParquetPageSourceFactory.java:415)', 'com.facebook.presto.hive.parquet.ParquetPageSourceFactory.getColumnType(ParquetPageSourceFactory.j

We can also save the output of our command as a Python variable!

In [11]:
result = %sql select * from retail.retail.czech limit 10
print(result)

Running query in 'presto://ezpresto-svc-locator.ezpresto.svc.cluster.local:8080'

select * from retail.retail.czech limit 10
RuntimeError: (pyhive.exc.DatabaseError) {'message': 'The column type of table retail.czech is declared as type string, but the Parquet file (file:/data/shared/retail-delta/data/czech/part-00000-90e5182a-79c2-4717-a5aa-6973936a287e-c000.snappy.parquet) declares the column as type INT32', 'errorCode': 16777224, 'errorName': 'HIVE_PARTITION_SCHEMA_MISMATCH', 'errorType': 'EXTERNAL', 'boolean': False, 'failureInfo': {'type': 'com.facebook.presto.spi.PrestoException', 'message': 'The column type of table retail.czech is declared as type string, but the Parquet file (file:/data/shared/retail-delta/data/czech/part-00000-90e5182a-79c2-4717-a5aa-6973936a287e-c000.snappy.parquet) declares the column as type INT32', 'suppressed': [], 'stack': ['com.facebook.presto.hive.parquet.ParquetPageSourceFactory.getParquetType(ParquetPageSourceFactory.java:415)', 'com.facebook.presto.hive.parquet.ParquetPageSourceFactory.getColumnType(ParquetPageSourceFactory.java